In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler


def split_data_with_window(x_in, y_in, split_window_size):
    # Initialize lists to store training and temporary sets
    x_out1_list, y_out1_list, x_out2_list, y_out2_list = [], [], [], []

    # Iterate through the data with the specified window size
    for i in range(0, len(x_in) - split_window_size, split_window_size + 1):
        x_out1_out2 = x_in.iloc[i:i+split_window_size+1]
        y_out1_out2 = y_in.iloc[i:i+split_window_size+1]

        # Separate the last row for the temporary set
        # [ :-1]: all elements except the last one
        # [-1:]:  selects only the last element
        # (:) is used to indicate slicing of a sequence
        # sequence[start : end : step]

        x_out1 = x_out1_out2.iloc[:-1]
        y_out1 = y_out1_out2.iloc[:-1]

        x_out2 = x_out1_out2.iloc[-1:]
        y_out2 = y_out1_out2.iloc[-1:]

        x_out1_list.append(x_out1)
        y_out1_list.append(y_out1)
        x_out2_list.append(x_out2)
        y_out2_list.append(y_out2)

    # Concatenate the lists into pandas DataFrames
    x_out1 = pd.concat(x_out1_list)
    y_out1 = pd.concat(y_out1_list)
    x_out2 = pd.concat(x_out2_list)
    y_out2 = pd.concat(y_out2_list)

    return x_out1, y_out1, x_out2, y_out2










# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

data = data.drop("date", axis=1)

data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()

data = data.astype('float32')


# Keep data until 31.08.2023
data = data.iloc[:10731]

#print(data['open'].dtype)
#print(data.shape)

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Now x_data and y_data are pandas DataFrames/Series, respectively

x_data.tail(1)



# Split Data to train and temp

# Define your split_window_size
split_window_size = 3

# Call the split_data_with_window function
x_train, y_train, x_temp, y_temp = split_data_with_window(x_data, y_data, split_window_size)


"""

# Print the last 5 rows of x_data
print("Last 5 rows of x_data:")
print(x_data.tail(5))

# Print the last 5 rows of x_train
print("\nLast 25 rows of x_train:")
print(x_train.tail(25))

print("\nLast 3 rows of y_train:")
print(y_temp.tail(3))

"""

# Split temp into val and test

# Define your split_window_size
split_window_size = 1

# Call the split_data_with_window function
x_val, y_val, x_test, y_test = split_data_with_window(x_temp, y_temp, split_window_size)




scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train)
x_val_normalized = scaler.transform(x_val)
x_test_normalized = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)


"""
# x_train_tensor inverse

x_test_original = scaler.inverse_transform(x_train_tensor.numpy())
print("\nFirst row of x_test_original:")
print(x_test_original[0])

print("\nFirst row of x_train:")
print(x_train.head(1))



print("\nLast row of x_test_original:")
print(x_test_original[-1])

print("\nLast row of x_train:")
print(x_train.tail(1))
"""









class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, learning_rate, window_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

In [10]:
import torch
import torch.nn as nn
import itertools
import random

# Define the hyperparameters to search over
input_sizes = [7]

hidden_sizes = [8]
num_layers_list = [2]

learning_rates = [0.0002]
window_sizes = [5]

num_epochs = 800
patience = 10 # Number of epochs to wait for improvement




# Set random seeds for reproducibility to achieve reproducibility in your PyTorch script
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) if torch.cuda.is_available() else None
np.random.seed(seed)
random.seed(seed)

# Additional steps to ensure determinism if needed !!!!!!!!!!!!!!

# ensures that cuDNN (CUDA Deep Neural Network library) will always produce the same results given the same input
torch.backends.cudnn.deterministic = True

# select the best algorithm for your input data. which can lead to faster execution times.
# different algorithms may be chosen even with the same input
torch.backends.cudnn.benchmark = False





# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Walk-forward validation training with sliding window for each hyperparameter combination
for hyperparams in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparams

    # Print hyperparameters
    print(f"Hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    # Initialize the model
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    counter = 0

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)


        # Early stopping based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

       # Calculate test loss after training is complete
    test_loss = 0.0
    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    print(f'Final Test Loss: {test_loss / len(x_test_tensor)}')

Hyperparameters: input_size=7, hidden_size=8, num_layers=2, learning_rate=0.0002, window_size=5


c:\VisualStudioCode\Repositories\Projectarbeit-Dow-Jones-Index\Projectarbeit-Dow-Jones-Index\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/800], Training Loss: 27106.72337908144, Validation Loss: 26555.455078125
Epoch [2/800], Training Loss: 25188.89507490649, Validation Loss: 24831.21484375
Epoch [3/800], Training Loss: 23704.539963062944, Validation Loss: 23454.642578125
Epoch [4/800], Training Loss: 22552.00051886824, Validation Loss: 22383.990234375
Epoch [5/800], Training Loss: 21673.043056203314, Validation Loss: 21562.671875
Epoch [6/800], Training Loss: 21004.204643214613, Validation Loss: 20932.716796875
Epoch [7/800], Training Loss: 19875.877832368238, Validation Loss: 19775.63671875
Epoch [8/800], Training Loss: 18845.673225793264, Validation Loss: 18777.5625
Epoch [9/800], Training Loss: 17961.15567246013, Validation Loss: 17899.828125
Epoch [10/800], Training Loss: 17142.866305356172, Validation Loss: 17102.705078125
Epoch [11/800], Training Loss: 16381.01566835584, Validation Loss: 16398.43359375
Epoch [12/800], Training Loss: 15673.672454281783, Validation Loss: 15700.9619140625
Epoch [13/800], Tra

In [2]:
import torch
import torch.nn as nn
import itertools
import random

# Define the hyperparameters to search over
input_sizes = [7]

hidden_sizes = [8]
num_layers_list = [2]

learning_rates = [0.0002]
window_sizes = [10]

num_epochs = 800
patience = 10 # Number of epochs to wait for improvement




# Set random seeds for reproducibility to achieve reproducibility in your PyTorch script
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) if torch.cuda.is_available() else None
np.random.seed(seed)
random.seed(seed)

# Additional steps to ensure determinism if needed !!!!!!!!!!!!!!

# ensures that cuDNN (CUDA Deep Neural Network library) will always produce the same results given the same input
torch.backends.cudnn.deterministic = True

# select the best algorithm for your input data. which can lead to faster execution times.
# different algorithms may be chosen even with the same input
torch.backends.cudnn.benchmark = False





# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Walk-forward validation training with sliding window for each hyperparameter combination
for hyperparams in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparams

    # Print hyperparameters
    print(f"Hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    # Initialize the model
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    counter = 0

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)


        # Early stopping based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

       # Calculate test loss after training is complete
    test_loss = 0.0
    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    print(f'Final Test Loss: {test_loss / len(x_test_tensor)}')

Hyperparameters: input_size=7, hidden_size=8, num_layers=2, learning_rate=0.0002, window_size=10


c:\VisualStudioCode\Repositories\Projectarbeit-Dow-Jones-Index\Projectarbeit-Dow-Jones-Index\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/800], Training Loss: 27139.816976187984, Validation Loss: 26678.49609375
Epoch [2/800], Training Loss: 25218.46366916535, Validation Loss: 24939.94140625
Epoch [3/800], Training Loss: 23729.2381552569, Validation Loss: 23550.349609375
Epoch [4/800], Training Loss: 22572.432634920475, Validation Loss: 22468.375
Epoch [5/800], Training Loss: 21689.941202375107, Validation Loss: 21637.515625
Epoch [6/800], Training Loss: 21018.21074385071, Validation Loss: 20999.607421875
Epoch [7/800], Training Loss: 20502.426546944876, Validation Loss: 20505.521484375
Epoch [8/800], Training Loss: 20101.030026548277, Validation Loss: 20118.076171875
Epoch [9/800], Training Loss: 19784.15340232882, Validation Loss: 19810.232421875
Epoch [10/800], Training Loss: 19530.681939495396, Validation Loss: 19562.65625
Epoch [11/800], Training Loss: 19325.491161030594, Validation Loss: 19361.291015625
Epoch [12/800], Training Loss: 19157.646084302054, Validation Loss: 19195.892578125
Epoch [13/800], Train

In [ ]:
l

In [ ]:
import torch
import torch.nn as nn
import itertools
import random

# Define the hyperparameters to search over
input_sizes = [7]

hidden_sizes = [8]
num_layers_list = [2]

learning_rates = [0.0002]
window_sizes = [1]

num_epochs = 800
patience = 10 # Number of epochs to wait for improvement




# Set random seeds for reproducibility to achieve reproducibility in your PyTorch script
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) if torch.cuda.is_available() else None
np.random.seed(seed)
random.seed(seed)

# Additional steps to ensure determinism if needed !!!!!!!!!!!!!!

# ensures that cuDNN (CUDA Deep Neural Network library) will always produce the same results given the same input
torch.backends.cudnn.deterministic = True

# select the best algorithm for your input data. which can lead to faster execution times.
# different algorithms may be chosen even with the same input
torch.backends.cudnn.benchmark = False





# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes))

# Walk-forward validation training with sliding window for each hyperparameter combination
for hyperparams in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size = hyperparams

    # Print hyperparameters
    print(f"Hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}")

    # Initialize the model
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    counter = 0

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i in range(len(x_train_tensor)):
            window_end = min(i + window_size, len(x_train_tensor))
            inputs = x_train_tensor[i:window_end].unsqueeze(0)
            labels = y_train_tensor[window_end - 1]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for i in range(len(x_val_tensor)):
                window_end = min(i + window_size, len(x_val_tensor))
                inputs = x_val_tensor[i:window_end].unsqueeze(0)
                labels = y_val_tensor[window_end - 1]

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)


        # Early stopping based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

       # Calculate test loss after training is complete
    test_loss = 0.0
    with torch.no_grad():
        for i in range(len(x_test_tensor)):
            window_end = min(i + window_size, len(x_test_tensor))
            inputs = x_test_tensor[i:window_end].unsqueeze(0)
            labels = y_test_tensor[window_end - 1]

            outputs = model(inputs)
            test_loss += criterion(outputs, labels)

    print(f'Final Test Loss: {test_loss / len(x_test_tensor)}')

Hyperparameters: input_size=7, hidden_size=8, num_layers=2, learning_rate=0.0002, window_size=1


c:\VisualStudioCode\Repositories\Projectarbeit-Dow-Jones-Index\Projectarbeit-Dow-Jones-Index\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/800], Training Loss: 27601.571313027154, Validation Loss: 27088.017578125
Epoch [2/800], Training Loss: 25974.36897974601, Validation Loss: 25621.724609375
Epoch [3/800], Training Loss: 24655.531628060326, Validation Loss: 24381.31640625
Epoch [4/800], Training Loss: 23559.339797727946, Validation Loss: 23351.0
Epoch [5/800], Training Loss: 22636.504643466895, Validation Loss: 22477.287109375
Epoch [6/800], Training Loss: 21644.373818423835, Validation Loss: 21486.556640625
Epoch [7/800], Training Loss: 20752.148981905106, Validation Loss: 20594.400390625
Epoch [8/800], Training Loss: 19920.51848053597, Validation Loss: 19778.43359375
Epoch [9/800], Training Loss: 19144.095796233356, Validation Loss: 19025.662109375
Epoch [10/800], Training Loss: 18418.885943635953, Validation Loss: 18308.8984375
Epoch [11/800], Training Loss: 17737.64538562916, Validation Loss: 17640.708984375
Epoch [12/800], Training Loss: 17096.105370754976, Validation Loss: 17013.814453125
Epoch [13/800], 